In [ ]:
pip install requests

In [ ]:
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

token_endpoint = 'https://icdaccessmanagement.who.int/connect/token'
client_id = 'your_client_id'
client_secret = 'your_client_secret'
scope = 'icdapi_access'
grant_type = 'client_credentials'


# get the OAUTH2 token

# set data to post
payload = {'client_id': client_id,
	   	   'client_secret': client_secret,
           'scope': scope,
           'grant_type': grant_type}

# make request
r = requests.post(token_endpoint, data=payload, verify=False).json()
token = r['access_token']


# access ICD API

uri = 'https://id.who.int/icd/entity/334423054'

# HTTP header fields to set
headers = {'Authorization':  'Bearer '+token,
           'Accept': 'application/json',
           'Accept-Language': 'en',
	   'API-Version': 'v2'}

# make request
r = requests.get(uri, headers=headers, verify=False)
print(r.text)
entity_data = r.json()

# print the result
#print (json.dumps(entity_data, indent=4))
figli=entity_data['child']
print()
#print(figli)
print(len(figli))
def GetDescription(figlio):
  additional='?include=diagnosticCriteria'
  url=figlio+additional
  print(url)
  r = requests.get(url, headers=headers, verify=False)
  entity_data = r.json()

  return entity_data

from bs4 import BeautifulSoup

def GetExtraDataFromBrowser(url):
    response = requests.get(url, headers=headers, verify=False)

    if response.status_code != 200:
        print(f"Errore: {response.status_code} per URL: {url}")
        return ""

    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup)
    # Estrai il titolo (esempio: da un <h1>)
    titolo = soup.find('h1').text.strip() if soup.find('h1') else 'Titolo non trovato'

    # Estrai i paragrafi di interesse
    paragrafi = [p.text.strip() for p in soup.find_all('p')]

    # Combina le informazioni
    extra_data = {
        "Titolo": titolo,
        "Descrizioni": " ".join(paragrafi)
    }

    return extra_data


def StampaFigli(figli,risultati):
  global limite_malattie
  global id
  if limite_malattie==0:
    return

  for child in figli:
    figlio=(GetDescription(child))
    urlSito=(figlio['browserUrl'])
    criteriDiagnostici=figlio['diagnosticCriteria'] if 'diagnosticCriteria' in figlio else ''
    criteriDiagnostici=criteriDiagnostici['@value'] if '@value' in  criteriDiagnostici else ''

    titolo=figlio['title'] if 'title' in figlio else ''
    titolo=titolo['@value'] if '@value' in titolo else ''
    descrizione=figlio['definition'] if 'definition' in figlio else ''
    descrizione = descrizione['@value'] if '@value' in descrizione else ''
    sinonimi=figlio['synonym'] if 'synonym'  in figlio else []
    sinonimi_trovati=''
    for sinonimo in sinonimi:
      sinonimo=sinonimo['label'] if 'label' in sinonimo else ''
      sinonimo=sinonimo['@value'] if '@value' in sinonimo else ''
      if sinonimo!='':
        sinonimi_trovati+=sinonimo+','
    # print(f"Nome: {titolo}\nDefinizione: {descrizione}\nSinonimi:{sinomi_trovati}\n--------------\n")
    if descrizione!='' or sinonimi_trovati!='':
          limite_malattie-=1
          print(limite_malattie)
          risultati.append({
            "ID_Malattia":id,
            "Nome": titolo,
            "Definizione": descrizione,
            "Sinonimi": sinonimi_trovati,
            "Diagnosi":criteriDiagnostici
            })
          id+=1
          if limite_malattie==0:
            break

    parenti=figlio['child'] if 'child' in figlio else ''
    print(figlio['@context'])
    if parenti!='' and limite_malattie>0:
      StampaFigli(parenti,risultati)

limite_malattie=150
id=0
risultati=[]
StampaFigli(figli,risultati)
# Salva i risultati in un file JSON
with open('ICD_ONTOLOGY.json', 'w', encoding='utf-8') as f:
    json.dump(risultati, f, ensure_ascii=False, indent=4)

{"@context":"http://id.who.int/icd/contexts/contextForFoundationEntity.json","@id":"http://id.who.int/icd/entity/334423054","parent":["http://id.who.int/icd/entity/455013390"],"child":["http://id.who.int/icd/entity/1516623224","http://id.who.int/icd/entity/405565289","http://id.who.int/icd/entity/486722075","http://id.who.int/icd/entity/76398729","http://id.who.int/icd/entity/1336943699","http://id.who.int/icd/entity/1321276661","http://id.who.int/icd/entity/991786158","http://id.who.int/icd/entity/108180424","http://id.who.int/icd/entity/1412387537","http://id.who.int/icd/entity/1884115764","http://id.who.int/icd/entity/794195577","http://id.who.int/icd/entity/1602669465","http://id.who.int/icd/entity/826065555","http://id.who.int/icd/entity/310393530","http://id.who.int/icd/entity/37291724","http://id.who.int/icd/entity/2110604642","http://id.who.int/icd/entity/430567349","http://id.who.int/icd/entity/213458094","http://id.who.int/icd/entity/882114523","http://id.who.int/icd/entity/5